# Test on principal / secondary home classification

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import compute_class_weight

In [ ]:
def load_data_file(n,r) :
    l_n = [15,73,150,1500,"a"]
    l_r = [50,"o"]
    if n not in l_n :
        print("n must be in "+str(l_n))
        return
    if r not in l_r :
        print("r must be in "+str(l_r))
        return
    data = np.load('./Data/Separated/train_test_n'+str(n)+'_r'+str(r)+'.npy',allow_pickle=True)
    return data[0],data[1],data[2],data[3]

In [ ]:
def confusion_matrix(y_test,y_est,name):
    
    #Handle of [-1,1] labels
    if np.min(y_est) == -1 :
        y_est_cl = (y_est + 1)/2
    else:
        y_est_cl = y_est
        
    if np.min(y_test) == -1 :
        y_test_cl = (y_test + 1)/2
    else:
        y_test_cl = y_test
    
    #Computing matrix
    matrix = [0,0,0,0] #TN,FN,FP,TP
    for test,est in zip(y_test_cl,y_est_cl):
        matrix[2*test+est] += 1
    np.save("Classifiers/results/matrix_"+name+".npy",np.array(matrix))
    return matrix

In [ ]:
def dispaly_confusion_matrix (y_test,y_est,name,matrix = None):
    if matrix is not None :
        matrix = confusion_matrix(y_test, y_est,name)
    print("TP : "+str(matrix [3])+"\t"+"FP : "+str(matrix[2]))
    print("FN : "+str(matrix [1])+"\t"+"TN : "+str(matrix[0]))

In [ ]:
def performance_evaluation(y_test,y_est,name):
    matrix = confusion_matrix(y_test,y_est,name)
    dispaly_confusion_matrix(y_test,y_est,name,matrix)
    TN,FN,FP,TP = matrix
    
    print("\n------------------\n")
    if TP+FP !=0 :
        precision = TP/(TP+FP)
        print("Precision RS : "+str(precision))
        recall = TP/(TP+FP)
        print("Recall : "+str(recall))
    else:
        print("Precision RS : None")
        print("Recall : None")
    if TN+FN !=0 :
        neg_pred_val = TN/(TN+FN)
        print("Precision RP : "+str(neg_pred_val))
    else:
        print("Precision RP : None")
    if TN+FP !=0 :
        specificity = TN/(TN+FP)
        print("Specificity : "+str(specificity))
    else :
        print("Specificity : None")
    accuracy = (TP+TN)/sum(matrix)
    print("Accuracy : "+str(accuracy))
    if (TP+FN)*(TN+FP) != 0:
        balanced_accuracy = (TP/(TP+FN)+TN/(TN+FP))/2
        print("Balanced Accuracy : "+str(balanced_accuracy))
    else:
        print("Balanced Accuracy : None")
    f1_score = 2*TP/(2*TP+FP+FN)
    print("F1 Score : "+str(f1_score))

In [ ]:
def PCA(x_train, x_test, n_components=10):
    """Perform PCA on image dataset keeping the top `n_components` components."""
    n_points_train = tf.gather(tf.shape(x_train), 0)
    n_points_test = tf.gather(tf.shape(x_test), 0)

    # Flatten to 1D
    x_train = tf.reshape(x_train, [n_points_train, -1])
    x_test = tf.reshape(x_test, [n_points_test, -1])

    # Normalize.
    feature_mean = tf.reduce_mean(x_train, axis=0)
    x_train_normalized = x_train - feature_mean
    x_test_normalized = x_test - feature_mean

    # Truncate.
    e_values, e_vectors = tf.linalg.eigh(tf.einsum('ji,jk->ik', x_train_normalized, x_train_normalized))
    return tf.einsum('ij,jk->ik', x_train_normalized, e_vectors[:,-n_components:]).numpy(),\
            tf.einsum('ij,jk->ik', x_test_normalized, e_vectors[:, -n_components:]).numpy()

# Random forest

In [ ]:
z_train, y_train, z_test, y_test = load_data_file(150,"o")
z_train_pca, z_test_pca = PCA(z_train, z_test,n_components=4)
z_train, y_train, z_test, y_test  = data[0],data[1],data[2],data[3]

clf = RandomForestClassifier(n_jobs=-1, max_depth=10, class_weight='balanced')

clf.fit(z_train, y_train)

pred = clf.predict(z_test)

In [ ]:
performance_evaluation(y_test,pred,'RF_150_pca_4')